In [3]:
def mask(e):
    label=[]
    for i in range(len(e)):
        l=None
        if (is_common_name(e[i], first_names) or is_common_name(e[i], first_names_lowcase))\
        and not (is_common_name(e[i], last_names) or is_common_name(e[i], last_names_lowcase)):
            if e[i][0].isupper():
                l='{First}'
            else:
                l='{first}'
        elif not (is_common_name(e[i], first_names) or is_common_name(e[i], first_names_lowcase))\
        and (is_common_name(e[i], last_names) or is_common_name(e[i], last_names_lowcase)): 
            if e[i][0].isupper():
                l='{Last}'
            else:
                l='{last}'
        elif e[i]=='.' or e[i]=='_' or e[i]=='-':
            l='{'+e[i]+'}'
        elif e[i][0].isdigit():
            l='{0-9}'
        else:
            if len(e[i])==1:
                l='{initial}'
                
        label.append(l)
    
    label1=[]
    for i in range(len(e)):
        if label[i]=='{initial}':
            if i==0:
                if label[i+1]=='{Last}' or label[i+1]=='{last}':
                    if e[i].isupper():
                        label[i]='{First initial}'
                    else:
                        label[i]='{first initial}'
                else:
                    if e[i].isupper():
                        label[i]='{Last initial}'
                    else:
                        label[i]='{last initial}'
            else:
                if label[i-1]=='{Fist}' or label[i-1]=='{first}':
                    if e[i].isupper():
                        label[i]='{Last initial}'
                    else:
                        label[i]='{last initial}'
                else:
                    if e[i].isupper():
                        label[i]='{First initial}'
                    else:
                        label[i]='{first initial}'    
                
        label1.append(label[i])
        
    return label1

In [4]:
import pandas as pd
import numpy as np

generic_mask=pd.read_csv('email_patterns.csv', header=None, usecols=[1,2,3,4,5,6])
new_header = generic_mask.iloc[0] #grab the first row for the header
generic_mask = generic_mask[1:] #take the data less the header row
generic_mask.columns = new_header #set the header row as the df header

emails = ['John.Smith@google.com', 
          'John_Smith@google.com', 
          'John-Smith@google.com',
          'JohnSmith123@google.com',
          'johnsmith@google.com',
          'Johnsmith123@google.com',
          'jsmith@google.com',
          'jsmith123@google.com',
          'johns@google.com',
          'john@google.com',
          'john123@google.com'
          'js@google.com',
          'johnsmith123@google.com']

first_names=['John', 'Nancy', 'Lori', 'Susan', 'Susanne', 'Mark']
first_names_lowcase=list(map(lambda x:x.lower(), first_names))
last_names=['Smith', 'Johnson', 'Jackson']
last_names_lowcase=list(map(lambda x:x.lower(), last_names))

def is_common_name(s,list_):
    if s in list_:
        return True
    else:
        return False  
    
def has_common_name_part(s, list_):
    for string in list_:
        if string in s:
            return string
    return None

mask_={}
for email in emails:
    s=email[:email.index('@')]
    chunks=[]
    for c in s:
        if c.isupper():
            chunks.append(c)
        elif c=='.' or c=='_' or c=='-':
            chunks.append(c)
        elif chunks:
                tmp=chunks.pop()
                if is_common_name(tmp, first_names) or\
                is_common_name(tmp, first_names_lowcase) or\
                is_common_name(tmp, last_names) or\
                is_common_name(tmp, last_names_lowcase):
                    chunks.append(tmp)
                    chunks.append(c)
                else:
                    if has_common_name_part(tmp, first_names):
                        tmp1=has_common_name_part(tmp, first_names)
                        chunks.append(tmp[:-len(tmp1)])
                        chunks.append(tmp1)
                    elif has_common_name_part(tmp, first_names_lowcase):
                        tmp1=has_common_name_part(tmp, first_names_lowcase)
                        chunks.append(tmp[:-len(tmp1)])
                        chunks.append(tmp1)
                    elif has_common_name_part(tmp, last_names):
                        tmp1=has_common_name_part(tmp, last_names)
                        chunks.append(tmp[:-len(tmp1)])
                        chunks.append(tmp1)
                    elif has_common_name_part(tmp, last_names_lowcase):
                        tmp1=has_common_name_part(tmp, last_names_lowcase)
                        chunks.append(tmp[:-len(tmp1)])
                        chunks.append(tmp1)
                    else:
                        if not c.isdigit():
                            chunks.append(tmp+c)
                        else:
                            chunks.append(tmp)
                        
                    if c.isdigit():
                        tmp=chunks.pop()
                        if tmp[-1].isdigit():
                            chunks.append(tmp+c)
                        else:
                            chunks.append(tmp)
                            chunks.append(c)
        
        else:
            chunks.append(c)
    if chunks:
        tmp=chunks.pop()
        if has_common_name_part(tmp, first_names):
            tmp1=has_common_name_part(tmp, first_names)
            if tmp[:-len(tmp1)]:
                chunks.append(tmp[:-len(tmp1)])
                chunks.append(tmp1)
            else:
                chunks.append(tmp)
        elif has_common_name_part(tmp, first_names_lowcase):
            tmp1=has_common_name_part(tmp, first_names_lowcase)
            if tmp[:-len(tmp1)]:
                chunks.append(tmp[:-len(tmp1)])
                chunks.append(tmp1)
            else:
                chunks.append(tmp)
        elif has_common_name_part(tmp, last_names):
            tmp1=has_common_name_part(tmp, last_names)
            if tmp[:-len(tmp1)]:
                chunks.append(tmp[:-len(tmp1)])
                chunks.append(tmp1)
            else:
                chunks.append(tmp)
        elif has_common_name_part(tmp, last_names_lowcase):
            tmp1=has_common_name_part(tmp, last_names_lowcase)
            if tmp[:-len(tmp1)]:
                chunks.append(tmp[:-len(tmp1)])
                chunks.append(tmp1)
            else:
                chunks.append(tmp)
        else:
            chunks.append(tmp)
          
#    print(email, chunks)
    list_=mask(chunks)
    for i in range(len(list_)):   
        mask_.setdefault(i, []).append(list_[i])
#print(mask_)
#print(generic_mask.shape)
mask_df= pd.DataFrame(0, index=range(generic_mask.shape[0]), columns=range(generic_mask.shape[1]))
#print(mask_df)
#print(generic_mask)
#for key, item in mask_.items():
for key1, item1 in generic_mask.iterrows():
      #  if key==key1:
        print('key=', key1, 'item=', item1[0], item1[1], item1[2], item1[3], item1[4], item1[5])
#    mask_df[key1]=item1
            
#print(mask_df)

key= 1 item= {} {} {} {} {} {}
key= 2 item= nan nan nan nan nan {0-9}
key= 3 item= nan {.] nan {.] nan nan
key= 4 item= nan {-} nan {-} nan nan
key= 5 item= nan {_} nan {_} nan nan
key= 6 item= {first} nan {first} nan {first} nan
key= 7 item= {last} nan {last} nan {last} nan
key= 8 item= {ifirst initial} nan {ifirst initial} nan {ifirst initial} nan
key= 9 item= {last initial} nan {last initial} nan {last initial} nan
key= 10 item= {First} nan {First} nan {First} nan
key= 11 item= {Last} nan {Last} nan {Last} nan
key= 12 item= {First initial} nan {First initial} nan {First initial} nan
key= 13 item= {Last initial} nan {Last initial} nan {Last initial} nan
key= 14 item= nan nan {middle initial} nan nan nan
key= 15 item= nan nan {middle} nan nan nan
key= 16 item= nan nan {second first} nan nan nan
key= 17 item= nan nan {patronimic} nan nan nan
key= 18 item= nan nan {Middle initial} nan nan nan
key= 19 item= nan nan {Middle} nan nan nan
key= 20 item= nan nan {Second first} nan nan nan
key